In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense


In [17]:
train_data = pd.read_csv("/kaggle/input/ai-for-social-good-aries-iitd-x-kaizen-24/Public/Task 1/train.csv")
test_data = pd.read_csv("/kaggle/input/ai-for-social-good-aries-iitd-x-kaizen-24/Public/Task 1/test.csv")
# print(train_data.head())


                                          tweet_text  claim
0  Coronavirus may have originated in lab linked ...      1
1  @SCMPNews China will buy all shares that's y i...      1
2  I'm curious if the pneumonia vaccine (Prevnar-...      1
3  dear you knew about covid 19 in january it is ...      1
4  Wow, they're as dumb as @realDonaldTrump sugge...      1


In [20]:
# Tokenize text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data['tweet_text'])
X = tokenizer.texts_to_sequences(train_data['tweet_text'])
X = pad_sequences(X, maxlen=100) # Assuming a maximum sequence length of 100
y = train_data['claim']

In [22]:
model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=100),
    Bidirectional(LSTM(64)),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_val, y_val))


Epoch 1/5
175/175 ━━━━━━━━━━━━━━━━━━━━ 20s 88ms/step - accuracy: 0.8522 - loss: 0.4139 - val_accuracy: 0.8791 - val_loss: 0.3432
Epoch 2/5
175/175 ━━━━━━━━━━━━━━━━━━━━ 20s 83ms/step - accuracy: 0.8868 - loss: 0.2706 - val_accuracy: 0.8562 - val_loss: 0.3863
Epoch 3/5
175/175 ━━━━━━━━━━━━━━━━━━━━ 20s 83ms/step - accuracy: 0.9584 - loss: 0.1153 - val_accuracy: 0.8405 - val_loss: 0.4743
Epoch 4/5
175/175 ━━━━━━━━━━━━━━━━━━━━ 21s 87ms/step - accuracy: 0.9848 - loss: 0.0456 - val_accuracy: 0.7568 - val_loss: 0.7205
Epoch 5/5
175/175 ━━━━━━━━━━━━━━━━━━━━ 20s 85ms/step - accuracy: 0.9948 - loss: 0.0257 - val_accuracy: 0.8026 - val_loss: 0.7549


In [24]:
# Predict probabilities
y_pred_prob = model.predict(X_val)
# Convert probabilities to binary predictions (0 or 1)
y_pred = np.round(y_pred_prob).flatten()
# Calculate accuracy
accuracy = accuracy_score(y_val, y_pred)
print("Validation Accuracy:", accuracy)



44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step
Validation Accuracy: 0.8025751072961373


In [27]:
# Make predictions
test_predictions = model.predict(X_test)
# Convert predictions to binary classes (0 or 1)
test_predictions = np.round(test_predictions).flatten()

# Create a DataFrame with the predictions
submission_df = pd.DataFrame({'claim': test_predictions})

# Optionally, if your test data has an 'ID' column, you can include it in the submission DataFrame
if 'ID' in test_data.columns:
    submission_df['ID'] = test_data['ID']

# Save predictions to a CSV file
submission_df.to_csv('submission.csv', index=False)



47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step
